In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record
from scipy.spatial.distance import cdist
from cellbgnet.analyze_eval import limited_matching, match_two_frames, recognition
from cellbgnet.analyze_eval import assemble_full_img_predictions, plot_full_img_predictions
from skimage.io import imread
import json


In [4]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/model_rotated_45_venus_beads2.pkl')

In [5]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [6]:
plot_train_record(chromo_model)

#### Real images as a function of z

In [7]:
data_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensity/')
save_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensityLocalizations/')
for pos in range(1, 6):
    images_dir = data_dir / Path('Pos' + str(pos).zfill(2)) / Path('Pos0')
    img_filenames = sorted(list(images_dir.glob('*.tif')))
    eval_imgs_all = []
    for filename in img_filenames:
        eval_imgs_all.append(imread(filename))
    eval_imgs_all = np.stack(eval_imgs_all)
    eval_imgs_all = eval_imgs_all[:, 1041:, :]
    print(f"Eval images shape: {eval_imgs_all.shape}")
    
    fov_size = [eval_imgs_all.shape[2] * 65, eval_imgs_all.shape[1] * 65]
    
    preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_imgs_all,
                                                 batch_size=16, use_tqdm=False,
                                                 nms=True, candidate_threshold=0.4,
                                                 nms_threshold=0.5, 
                                                 pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                                 plot_num=1,
                                                 win_size=128,
                                                 padding=True,
                                                 start_field_pos=[0, 0],
                                                 padded_background=chromo_model.evaluation_params['padded_background'])
    
    preds_tmp_np = np.array(preds_tmp)
    frame_wise_xyz = [preds_tmp_np[np.where(preds_tmp_np[:, 1] == i)].tolist() for i in range(1, 1 + eval_imgs_all.shape[0])]
    
    bead_data = []
    for beadno in range(len(frame_wise_xyz[0])):
        single_bead_data = {}
        single_bead_data['bead_no'] = beadno
        single_bead_data['x'] = [frame_wise_xyz[0][beadno][2]]
        single_bead_data['y'] = [frame_wise_xyz[0][beadno][3]]
        single_bead_data['z'] = [frame_wise_xyz[0][beadno][4]]
        single_bead_data['ph'] = [frame_wise_xyz[0][beadno][5]]
        single_bead_data['x_sigma'] = [frame_wise_xyz[0][beadno][7]]
        single_bead_data['y_sigma'] = [frame_wise_xyz[0][beadno][8]]
        single_bead_data['z_sigma'] = [frame_wise_xyz[0][beadno][9]]
        single_bead_data['ph_sigma'] = [frame_wise_xyz[0][beadno][10]]
        for i in range(1, eval_imgs_all.shape[0]):
            perf_dict, matches = match_two_frames([frame_wise_xyz[0][beadno]], frame_wise_xyz[i], min_int=0, limited_x=[0, fov_size[0]],
                                         limited_y=[0, fov_size[1]], border=450, print_res=False, tolerance=250,
                                         tolerance_ax=np.inf)
            if len(matches) == 1:
                single_bead_data['x'].append(matches[0, 4])
                single_bead_data['y'].append(matches[0, 5])
                single_bead_data['z'].append(matches[0, 6])
                single_bead_data['ph'].append(matches[0, 7])
                single_bead_data['x_sigma'].append(matches[0, 8])
                single_bead_data['y_sigma'].append(matches[0, 9])
                single_bead_data['z_sigma'].append(matches[0, 10])
                single_bead_data['ph_sigma'].append(matches[0, 11])
                #print(f"First bead x: {matches[:, 0]} -- matched: {matches[:, 4]}")
                #print(f"First bead y: {matches[:, 1]} -- matched: {matches[:, 5]}")
                #print(f"First bead z: {matches[:, 2]} -- matched: {matches[:, 6]}")
                #print(f"First bead ph: {matches[:, 3]} -- matched: {matches[:, 7]}")
                #print("---------")
            else:
                print(f"For beadno: {beadno} --- Skipping frame {i} --- :(")
        bead_data.append(single_bead_data)
    
    with open(save_dir /  Path('Pos' + str(pos)), 'w') as fout:
        json.dump(bead_data, fout)
    print(f" Position: {pos} -- Done")

Eval images shape: (200, 1041, 1302)
processing area:1/99, input field_xy:[ -2 145  -3 144], use_coordconv:True, retain locs in area:[-2, 125, -3, 124]

/home/pk/Documents/cellbgnet/cellbgnet/utils/hardware.py:9: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  return torch.cuda.FloatTensor(x.astype('float32'))


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
matches is empty!
For beadno: 0 --- Skipping frame 79 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 118 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 151 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 153 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 197 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 2 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 4 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 16 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 19 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 20 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 23 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 26 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 29 --- :(
matches is empty!
For beadno: 2 --- Skipping frame 34 --- :(
matches is empty!


/home/pk/Documents/cellbgnet/cellbgnet/analyze_eval.py:1318: RuntimeWarning: invalid value encountered in scalar divide
  jor = 100 * jaccard / rmse_lat


 Position: 1 -- Done
Eval images shape: (200, 1041, 1302)
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
matches is empty!
For beadno: 0 --- Skipping frame 5 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 8 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 10 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 11 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 12 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 15 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 22 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 27 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 44 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 45 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 60 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 62 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 71 --- :(
matches is empty!
For bea

In [8]:
1+1

2

In [9]:
data_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/zStack_locError_101/')
save_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/localizations_vs_z/')
try:
    for pos in range(100, 150):
        images_dir = data_dir / Path('Pos' + str(pos))
        img_filenames = sorted(list(images_dir.glob('*.tif')))
        eval_imgs_all = []
        for filename in img_filenames:
            eval_imgs_all.append(imread(filename))
        eval_imgs_all = np.stack(eval_imgs_all)
        eval_imgs_all = eval_imgs_all[:, 1041:, :]
        print(f"Eval images shape: {eval_imgs_all.shape}")
        
        fov_size = [eval_imgs_all.shape[2] * 65, eval_imgs_all.shape[1] * 65]
        
        preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_imgs_all,
                                                     batch_size=16, use_tqdm=False,
                                                     nms=True, candidate_threshold=0.4,
                                                     nms_threshold=0.5, 
                                                     pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                                     plot_num=1,
                                                     win_size=128,
                                                     padding=True,
                                                     start_field_pos=[0, 0],
                                                     padded_background=chromo_model.evaluation_params['padded_background'])
        
        preds_tmp_np = np.array(preds_tmp)
        frame_wise_xyz = [preds_tmp_np[np.where(preds_tmp_np[:, 1] == i)].tolist() for i in range(1, 1 + eval_imgs_all.shape[0])]
        
        bead_data = []
        for beadno in range(len(frame_wise_xyz[0])):
            single_bead_data = {}
            single_bead_data['bead_no'] = beadno
            single_bead_data['x'] = [frame_wise_xyz[0][beadno][2]]
            single_bead_data['y'] = [frame_wise_xyz[0][beadno][3]]
            single_bead_data['z'] = [frame_wise_xyz[0][beadno][4]]
            single_bead_data['ph'] = [frame_wise_xyz[0][beadno][5]]
            single_bead_data['x_sigma'] = [frame_wise_xyz[0][beadno][7]]
            single_bead_data['y_sigma'] = [frame_wise_xyz[0][beadno][8]]
            single_bead_data['z_sigma'] = [frame_wise_xyz[0][beadno][9]]
            single_bead_data['ph_sigma'] = [frame_wise_xyz[0][beadno][10]]
            for i in range(1, eval_imgs_all.shape[0]):
                perf_dict, matches = match_two_frames([frame_wise_xyz[0][beadno]], frame_wise_xyz[i], min_int=0, limited_x=[0, fov_size[0]],
                                             limited_y=[0, fov_size[1]], border=450, print_res=False, tolerance=250,
                                             tolerance_ax=np.inf)
                if len(matches) == 1:
                    single_bead_data['x'].append(matches[0, 4])
                    single_bead_data['y'].append(matches[0, 5])
                    single_bead_data['z'].append(matches[0, 6])
                    single_bead_data['ph'].append(matches[0, 7])
                    single_bead_data['x_sigma'].append(matches[0, 8])
                    single_bead_data['y_sigma'].append(matches[0, 9])
                    single_bead_data['z_sigma'].append(matches[0, 10])
                    single_bead_data['ph_sigma'].append(matches[0, 11])
                    #print(f"First bead x: {matches[:, 0]} -- matched: {matches[:, 4]}")
                    #print(f"First bead y: {matches[:, 1]} -- matched: {matches[:, 5]}")
                    #print(f"First bead z: {matches[:, 2]} -- matched: {matches[:, 6]}")
                    #print(f"First bead ph: {matches[:, 3]} -- matched: {matches[:, 7]}")
                    #print("---------")
                else:
                    print(f"For beadno: {beadno} --- Skipping frame {i} --- :(")
            bead_data.append(single_bead_data)
        
        with open(save_dir /  Path('Pos' + str(pos)), 'w') as fout:
            json.dump(bead_data, fout)
        print(f" Position: {pos} -- Done")
except Exception as e:
    print(f"LOL some exception {e}")

Eval images shape: (50, 1041, 1302)
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
matches is empty!
For beadno: 0 --- Skipping frame 1 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 2 --- :(
For beadno: 0 --- Skipping frame 6 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 7 --- :(
For beadno: 0 --- Skipping frame 8 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 9 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 14 --- :(
For beadno: 0 --- Skipping frame 16 --- :(
For beadno: 0 --- Skipping frame 18 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 19 --- :(
For beadno: 0 --- Skipping frame 21 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 22 --- :(
For beadno: 0 --- Skipping frame 23 --- :(
For beadno: 0 --- Skipping frame 25 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 26 --- :(
matches is empty!
For beadno: 0 --- Skipping frame 27 -

In [10]:
1+1

2

In [17]:
data_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/zStack_locError_103/')
save_dir = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/localizations_vs_z_103/')

for pos in range(100, 150):
    try:
        images_dir = data_dir / Path('Pos' + str(pos))
        img_filenames = sorted(list(images_dir.glob('*.tif')))
        eval_imgs_all = []
        for filename in img_filenames:
            eval_imgs_all.append(imread(filename))
        eval_imgs_all = np.stack(eval_imgs_all)
        eval_imgs_all = eval_imgs_all[:, 1041:, :]
        print(f"Eval images shape: {eval_imgs_all.shape}")
        
        fov_size = [eval_imgs_all.shape[2] * 65, eval_imgs_all.shape[1] * 65]
            
        preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_imgs_all,
                                                     batch_size=16, use_tqdm=False,
                                                     nms=True, candidate_threshold=0.4,
                                                     nms_threshold=0.5, 
                                                     pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                                     plot_num=1,
                                                     win_size=128,
                                                     padding=True,
                                                     start_field_pos=[0, 0],
                                                     padded_background=chromo_model.evaluation_params['padded_background'])
        
        preds_tmp_np = np.array(preds_tmp)
        if len(preds_tmp_np) == 0:
            frame_wise_xyz = []
        else:
            frame_wise_xyz = [preds_tmp_np[np.where(preds_tmp_np[:, 1] == i)].tolist() for i in range(1, 1 + eval_imgs_all.shape[0])]
        
        bead_data = []
        if len(frame_wise_xyz) != 0:
            for beadno in range(len(frame_wise_xyz[0])):
                single_bead_data = {}
                single_bead_data['bead_no'] = beadno
                single_bead_data['x'] = [frame_wise_xyz[0][beadno][2]]
                single_bead_data['y'] = [frame_wise_xyz[0][beadno][3]]
                single_bead_data['z'] = [frame_wise_xyz[0][beadno][4]]
                single_bead_data['ph'] = [frame_wise_xyz[0][beadno][5]]
                single_bead_data['x_sigma'] = [frame_wise_xyz[0][beadno][7]]
                single_bead_data['y_sigma'] = [frame_wise_xyz[0][beadno][8]]
                single_bead_data['z_sigma'] = [frame_wise_xyz[0][beadno][9]]
                single_bead_data['ph_sigma'] = [frame_wise_xyz[0][beadno][10]]
                for i in range(1, eval_imgs_all.shape[0]):
                    perf_dict, matches = match_two_frames([frame_wise_xyz[0][beadno]], frame_wise_xyz[i], min_int=0, limited_x=[0, fov_size[0]],
                                                 limited_y=[0, fov_size[1]], border=450, print_res=False, tolerance=250,
                                                 tolerance_ax=np.inf)
                    if len(matches) == 1:
                        single_bead_data['x'].append(matches[0, 4])
                        single_bead_data['y'].append(matches[0, 5])
                        single_bead_data['z'].append(matches[0, 6])
                        single_bead_data['ph'].append(matches[0, 7])
                        single_bead_data['x_sigma'].append(matches[0, 8])
                        single_bead_data['y_sigma'].append(matches[0, 9])
                        single_bead_data['z_sigma'].append(matches[0, 10])
                        single_bead_data['ph_sigma'].append(matches[0, 11])
                        #print(f"First bead x: {matches[:, 0]} -- matched: {matches[:, 4]}")
                        #print(f"First bead y: {matches[:, 1]} -- matched: {matches[:, 5]}")
                        #print(f"First bead z: {matches[:, 2]} -- matched: {matches[:, 6]}")
                        #print(f"First bead ph: {matches[:, 3]} -- matched: {matches[:, 7]}")
                        #print("---------")
                    else:
                        print(f"For beadno: {beadno} --- Skipping frame {i} --- :(")
                bead_data.append(single_bead_data)
        
        with open(save_dir /  Path('Pos' + str(pos)), 'w') as fout:
            json.dump(bead_data, fout)
        print(f" Position: {pos} -- Done")
    except Exception as e:
        print(f"LOL some exception {e}")

Eval images shape: (50, 1041, 1302)
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
 Position: 100 -- Done
Eval images shape: (50, 1041, 1302)
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
 Position: 101 -- Done
Eval images shape: (50, 1041, 1302)
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
 Position: 102 -- Done
Eval images shape: (50, 1041, 1302)
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
 Position: 103 -- Done
Eval images shape: (50, 1041, 1302)
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]
 Position: 104 -- Done
Eval images shape: (50, 1041, 1302)
processing area:99/99, input field_xy:[

In [14]:
1+1

2